# Create weights for OM4p25

In [1]:
import xarray as xr
import xesmf
import numpy as np

In [2]:
erainterim_indir = '/Volumes/P4/workdir/raphael/ERAinterim/nc_processed/'

In [3]:
erainterim_outdir = '/Volumes/P4/workdir/raphael/ERAinterim/nc_processed_OM4p25/'

## Building the grids

In [7]:
# ERA-interim
erai_grid = xr.open_dataset('/Volumes/L4/ERAinterim/erai_grid.nc')
#erai_grid = xr.open_dataset(f'{erainterim_indir}/precip_ERAinterim_1979_daily_ROMS.nc', decode_times=False, drop_variables=['rain', 'rain_time'])

lon = erai_grid['lon'].values
lon_bnds = np.concatenate((np.array([lon[0] -0.5 * 0.7031]), 0.5 * (lon[:-1] + lon[1:]), np.array([lon[-1] + 0.5 * 0.7031])), axis=0)

lat = erai_grid['lat'].values
lat_bnds = np.concatenate((np.array([-90]), 0.5 * (lat[:-1] + lat[1:]), np.array([90])), axis=0)

erai_grid['lon_b'] = xr.DataArray(data=lon_bnds, dims=('lonp1'))
erai_grid['lat_b'] = xr.DataArray(data=lat_bnds, dims=('latp1'))

In [8]:
erai_grid

<xarray.Dataset>
Dimensions:  (lat: 256, latp1: 257, lon: 512, lonp1: 513)
Coordinates:
  * lon      (lon) float64 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3
  * lat      (lat) float64 -89.46 -88.77 -88.07 -87.37 ... 88.07 88.77 89.46
Dimensions without coordinates: latp1, lonp1
Data variables:
    lon_b    (lonp1) float64 -0.3515 0.3516 1.055 1.758 ... 358.2 358.9 359.6
    lat_b    (latp1) float64 -90.0 -89.11 -88.42 -87.72 ... 88.42 89.11 90.0

In [9]:
# OM4p5
om4_supergrid = xr.open_dataset('/Users/raphael/STORAGE/MOM6/OM4_05/ocean_hgrid.nc')

In [10]:
om4_supergrid

<xarray.Dataset>
Dimensions:   (nx: 1440, nxp: 1441, ny: 1152, nyp: 1153)
Dimensions without coordinates: nx, nxp, ny, nyp
Data variables:
    tile      |S255 ...
    y         (nyp, nxp) float64 ...
    x         (nyp, nxp) float64 ...
    dy        (ny, nxp) float64 ...
    dx        (nyp, nx) float64 ...
    area      (ny, nx) float64 ...
    angle_dx  (nyp, nxp) float64 ...

In [11]:
om4_grid = xr.Dataset()
lon = om4_supergrid['x'].values[1::2,1::2].copy()
lat = om4_supergrid['y'].values[1::2,1::2].copy()
lon_b = om4_supergrid['x'].values[0::2,0::2].copy()
lat_b = om4_supergrid['y'].values[0::2,0::2].copy()

om4_grid['lon'] = xr.DataArray(data=lon, dims=('y', 'x'))
om4_grid['lat'] = xr.DataArray(data=lat, dims=('y', 'x'))
om4_grid['lon_b'] = xr.DataArray(data=lon_b, dims=('yp1', 'xp1'))
om4_grid['lat_b'] = xr.DataArray(data=lat_b, dims=('yp1', 'xp1'))

In [12]:
om4_grid

<xarray.Dataset>
Dimensions:  (x: 720, xp1: 721, y: 576, yp1: 577)
Dimensions without coordinates: x, xp1, y, yp1
Data variables:
    lon      (y, x) float64 -299.8 -299.2 -298.8 -298.2 ... 59.99 59.99 60.0
    lat      (y, x) float64 -77.91 -77.91 -77.91 -77.91 ... 65.39 65.18 64.97
    lon_b    (yp1, xp1) float64 -300.0 -299.5 -299.0 -298.5 ... 60.0 60.0 60.0
    lat_b    (yp1, xp1) float64 -78.0 -78.0 -78.0 -78.0 ... 65.29 65.08 64.87

In [ ]:
%time regrid_conserve = xesmf.Regridder(erai_grid, om4_grid, method='conservative', periodic=True)

Create weight file: conservative_256x512_576x720.nc


In [ ]:
%time regrid_patch = xesmf.Regridder(erai_grid, om4_grid, method='patch', periodic=True)

In [ ]:
%time regrid_bilin = xesmf.Regridder(erai_grid, om4_grid, method='bilinear', periodic=True)